**Image classification with your own data**
---

Before running code, make sure the training images are in the class directories for train, valid and test.

1) Train directory is './drive/My Drive/public/train'

2) Valid directory is './drive/My Drive/public/valid'

3) Test directory is './drive/My Drive/public/test'

For example, with 'HDH' and 'OH' classes 
* ./drive/My Drive/public/train/HDH/*.jpg
* ./drive/My Drive/public/train/OH/*.jpg
* ./drive/My Drive/public/valid/HDH/*.jpg
* ./drive/My Drive/public/valid/OH/*.jpg
* ./drive/My Drive/public/test/HDH/*.jpg
* ./drive/My Drive/public/test/OH/*.jpg

Initial tutorial code was provided by hchoi@handong.edu, Nov. 30, 2019. And the code is updated in Jun. 23. 2020 for Gcamp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import os
import warnings
warnings.filterwarnings("ignore")

# try different network architectures
* different kernel size and numbers

In [ ]:
class MyCNN(nn.Module):
    def __init__(self, output_dim=10):
        super(MyCNN,self).__init__()

        self.output_dim=output_dim

        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3,32,3,padding=1), # try with different kernels
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32,32,3,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(4,4), # 32 x (25x25)
            
            nn.Conv2d(32,16,3,padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16,16,3,padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(5,5) # 16 x (5x5) 
        )
        conv_size = self.get_conv_size(3, input_size)
        self.fc_layer = nn.Sequential(
            nn.Linear(conv_size,100),
            nn.BatchNorm1d(100),
            nn.ReLU(),
            nn.Linear(100,output_dim)
        )       

    def get_conv_size(self, channel, shape):
        o = self.cnn_layers(torch.zeros(1, channel, *shape))
        return int(np.prod(o.size()))
        
    def forward(self,x):
        batch_size, c, h, w = x.data.size()
        out = self.cnn_layers(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        return out



# try different learning rates

In [ ]:
learning_rate = 0.0005
output_dim=5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


**data loader**

In [ ]:
resize=(120, 120)
input_size=(100, 100)

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(resize),
        transforms.RandomCrop(input_size), # data augmentation
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.3, 0.3, 0.3])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(resize),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.3, 0.3, 0.3])
    ]),
}
test_transform = data_transforms['valid']

In [ ]:
batch_size = 64 # try different batch_size values
data_dir = './drive/My Drive/public/'
train_dir = 'train'
valid_dir = 'valid1'

train_set = datasets.ImageFolder(data_dir+train_dir, data_transforms['train'])
valid_set = datasets.ImageFolder(data_dir+valid_dir, data_transforms['valid'])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                              shuffle=True, num_workers=4)          
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size,
                                              shuffle=True, num_workers=4)

train_size = len(train_set)
valid_size = len(valid_set)

class_names = train_set.classes

print(class_names) 
print(f'Train image size: {train_size}')
print(f'Validation image size: {valid_size}')

['ANH', 'HDH', 'Hyoam', 'NTH', 'OH']
Train image size: 4186
Validation image size: 241


**make model instance, optimizer instance, and train the model**

In [ ]:
model = MyCNN(output_dim=output_dim).to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

param_list = list(model.children())
print(param_list)

[Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
), Sequential(
  (0): Linear(in_features=400, out_features=100, bias=True)
  (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, 

**training**
* implement 'early stopping' 

In [ ]:
result_dir = 'drive/My Drive/public/results/'
num_epoch = 3 # try with different epochs and find the best epoch

if not os.path.exists(result_dir):
    os.mkdir(result_dir)    
    
for i in range(num_epoch):
    model.train()
    # make a training loop of the model
    for j, [image,label] in enumerate(train_loader):
        # x = ?
        # y_= ?
        
        optimizer.zero_grad()
        
        # output = ?
        # loss = ?
        
        loss.backward()
        optimizer.step()
        
        #For Logging
        #if j % 30 == 0:
        #   print(i,j, loss.data.cpu())
    
    model.eval()
    hits = 0
    for k,[image,label] in enumerate(valid_loader):
        x = image.to(device)
        y_= label.to(device)

        output = model(x)
        y_est = output.argmax(1)
        hits = hits + sum(y_est == y_).cpu()
    print('Epochs', i, 'Hits', int(hits), 'Accuracy', float(hits/(valid_size+0.0)))    

torch.save(model, result_dir + 'teamX.model')
print('training is done by max_epochs', num_epoch)

NameError: ignored


---
training is done! 

---

**you can test your model**

In [ ]:
test_batch_size = 10
result_dir = 'drive/My Drive/public/results/'
model_name = 'teamX.model'

model = torch.load(result_dir + model_name)
model.to(device)
model.eval()

test_dir = './drive/My Drive/public/valid2'
test_set = datasets.ImageFolder(test_dir, test_transform)
              
test_loader = torch.utils.data.DataLoader(test_set, batch_size=test_batch_size,
                                          shuffle=False, num_workers=4)

hits = 0
for k,[image,label] in enumerate(test_loader):
    x = image.to(device)
    y_= label.to(device)
  
    output = model(x)
    y_est = output.argmax(1)
    print('Target', label.numpy(), 'Prediction', y_est.cpu().numpy())
    hits = hits + sum(y_est == y_)
print('hits', int(hits),'accuracy', float(hits/(len(test_set)+0.0)))

Target [0 0 0 0 0 0 0 0 0 0] Prediction [0 0 0 0 0 0 0 0 0 0]
Target [0 0 0 0 0 0 0 0 0 0] Prediction [0 0 0 0 0 0 0 0 0 0]
Target [0 0 0 0 0 0 0 0 0 0] Prediction [0 0 0 0 0 0 0 0 0 0]
Target [0 0 0 0 0 0 0 0 0 0] Prediction [0 0 0 0 0 0 0 0 0 0]
Target [1 1 1 1 1 1 1 1 1 1] Prediction [1 1 1 1 1 1 1 1 1 1]
Target [1 1 1 1 1 1 1 1 1 1] Prediction [1 1 1 1 1 1 1 1 1 1]
Target [1 1 1 1 1 1 1 1 1 1] Prediction [1 1 1 1 1 0 0 0 0 0]
Target [1 1 1 1 1 1 1 1 1 1] Prediction [0 0 0 0 4 0 1 4 1 1]
Target [2 2 2 2 2 2 2 2 2 2] Prediction [2 2 2 2 2 2 2 2 2 2]
Target [2 2 2 2 2 2 2 2 2 2] Prediction [2 2 2 2 2 2 2 4 2 2]
Target [2 2 2 2 2 2 2 2 2 2] Prediction [2 2 2 2 2 2 2 0 0 2]
Target [2 2 2 2 2 2 2 2 2 2] Prediction [2 2 2 0 0 4 0 3 0 0]
Target [3 3 3 3 3 3 3 3 3 3] Prediction [3 3 3 3 3 3 3 3 3 3]
Target [3 3 3 3 3 3 3 3 3 3] Prediction [3 3 3 3 3 3 3 3 3 3]
Target [3 3 3 3 3 3 3 3 3 3] Prediction [3 3 3 3 3 3 4 4 3 4]
Target [3 3 3 3 3 3 3 3 3 3] Prediction [4 4 4 4 4 0 3 4 4 4]
Target [

# classify one image

In [ ]:
from skimage import io

img_name = './drive/My Drive/public/test/test1.jpg'
test_img = io.imread(img_name)
test_img = transforms.ToPILImage()(test_img)
test_img = test_transform(test_img)
test_data = test_img.unsqueeze(0).to(device)

output=model(test_data)

class_id = output.argmax(dim=1).cpu().numpy()[0]
print(img_name.split('/')[-1], '==>', class_id, class_names[class_id])

test1.jpg ==> 0 ANH


# classify all images in a directory

In [ ]:
from skimage import io
import glob

print('please print?')
img_dir = 'drive/My Drive/public/test/'
file_list = glob.glob(img_dir + '*.*')
for img_name in file_list:
  test_img = io.imread(img_name)
  test_img = transforms.ToPILImage()(test_img)
  test_img = test_transform(test_img)
  test_data = test_img.unsqueeze(0).to(device)

  output=model(test_data)

  class_id = output.argmax(dim=1).cpu().numpy()[0]
  print(img_name.split('/')[-1], '==>', class_id, class_names[class_id])


please print?
test1.jpg ==> 0 ANH
D10_ANH09.jpg ==> 0 ANH
D06_ANH32.jpg ==> 0 ANH
D07_ANH22.jpg ==> 0 ANH
D08_ANH23.jpg ==> 0 ANH
D10_ANH46.jpg ==> 0 ANH
D12_AHN36.jpg ==> 0 ANH
D07_HDH17.jpg ==> 0 ANH
D13_ANH01.jpg ==> 0 ANH
D13_ANH45.jpg ==> 0 ANH
D06_HDH08.jpg ==> 1 HDH
D07_HDH07.jpg ==> 1 HDH
D02_HDH50.jpg ==> 1 HDH
D18_ANH48.jpg ==> 0 ANH
D15_HDH04.JPG ==> 1 HDH
D13_ANH47.jpg ==> 0 ANH
D07_HDH30.jpg ==> 1 HDH
D07_HDH45.jpg ==> 1 HDH
D09_HDH31.JPG ==> 1 HDH
D14_Hyoam12.jpg ==> 2 Hyoam
D07_HDH49.jpg ==> 1 HDH
D18_Hyoam40.JPG ==> 2 Hyoam
D12_HDH10.jpg ==> 1 HDH
D15_Hyoam47.jpg ==> 2 Hyoam
D08_Hyoam09.jpg ==> 2 Hyoam
D11_Hyoam05.JPG ==> 2 Hyoam
D09_Hyoam09.JPG ==> 2 Hyoam
D09_Hyoam31.JPG ==> 2 Hyoam
D01_Hyoam41.jpg ==> 2 Hyoam
D17_Hyoam39.jpeg ==> 2 Hyoam
D18_Hyoam38.JPG ==> 2 Hyoam
D13_NTH30.jpg ==> 3 NTH
D05_NTH31.jpg ==> 3 NTH
D05_NTH03.jpg ==> 3 NTH
D05_NTH45.jpg ==> 3 NTH
D17_NTH08.jpg ==> 3 NTH
D11_NTH30.jpeg ==> 3 NTH
D11_NTH01.jpeg ==> 3 NTH
D09_NTH30.jpg ==> 3 NTH
D11_NTH15.j

KeyboardInterrupt: ignored

The end! (hchoi@handong.edu)

---

